<a href="https://colab.research.google.com/github/Qisam0322/Prediction-of-Product-Sales/blob/main/Product_Sales_Preprocessing_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Product Sales

- SAM QI

#Preprocesing for Machine Learning

## Imports and load fresh data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer,make_column_transformer,make_column_selector
from sklearn import set_config
set_config(transform_output='pandas')

In [3]:
#Load fresh data
fname2='/content/drive/MyDrive/CodingDojo/02-IntroML/Week05/Data/sales_predictions_2023.csv'
df=pd.read_csv(fname2)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
df.shape

(8523, 12)

## Performing Preprocessing Steps

In [6]:
# Check duplicates
df.duplicated().sum()

0

In [7]:
# Check and fix inconsistancies
string_cols=df.select_dtypes('object').columns
for col in string_cols:
  print(f'Value count for {col}')
  print(df[col].value_counts())
  print('\n')

Value count for Item_Identifier
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64


Value count for Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64


Value count for Item_Type
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64


Value count for Outlet_Identifier
OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT0

In [8]:
#Fix inconsistancies in 'Item_Fat_Content' column
df['Item_Fat_Content']=df['Item_Fat_Content'].replace({'LF':'Low Fat','low fat':'Low Fat','reg':'Regular'})
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [11]:
#Check features data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Features data types are correct.

In [9]:
#Check duplicates
df.duplicated().sum()

0

No duplicates found.

In [10]:
#Check for impossible values in numercial columns
df.describe(include='number')

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


No impossible value found

In [17]:
#Check missing values
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Found missing values in 'Item Weight' and 'Outlet Size'. Null values will be addressed after train test split to ensure no data leakage.

## Identify the features (X) and target (y)

In [18]:
#Identify X and y, drop the "Item_Identifier" feature
#because it has very high cardinality.
target='Item_Outlet_Sales'
X=df.drop(columns=target).drop(columns='Item_Identifier').copy()
y = df[target].copy()
X.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store
4,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1


## Perform train test split

In [20]:
#Perform train test split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

## Create a preprocessing object to prepare the dataset for Machine Learning

### Create Simpleimputer to fill nulls

In [21]:
#Identify missing values
X_train.isna().sum()

Item_Weight                  1107
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1812
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [22]:
# Filling missing value for numerical column 'Item_Weight',
# upon further analyzing, nulls are missing at random.
# Decided to use 'median' to fill null values.
impute_median=SimpleImputer(strategy='median')

In [23]:
#Filling missing value for category(ordinal) column 'Outlet_Size'
#Decided to use 'MISSING' to fill nulls.
impute_na_ord=SimpleImputer(strategy='constant',fill_value='NA')

###Create preprocessing object for numeric columns

In [24]:
#define numeric columns
num_cols=X_train.select_dtypes('number').columns
num_cols

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')

In [25]:
#Create a StandardScaler
num_scaler=StandardScaler()

In [26]:
#create a pipline for numeric column
num_pip=make_pipeline(impute_median,num_scaler)
num_pip

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [27]:
#Create a tuple for each transformer
num_tuple=('numeric',num_pip,num_cols)
num_tuple

('numeric',
 Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                 ('standardscaler', StandardScaler())]),
 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
        'Outlet_Establishment_Year'],
       dtype='object'))

###Create preprocessing object for ordinal columns

In [28]:
#define ordinal columns
ord_cols=['Outlet_Size']
ord_cols

['Outlet_Size']

In [29]:
#Create a StandardScaler
ord_scaler=StandardScaler()

In [30]:
#Create ordinal encoder
size_order=['NA','Small','Medium','High']
ordinal_category_orders=[size_order]
ord_encoder=OrdinalEncoder(categories=ordinal_category_orders)

In [31]:
#create a pipline for ordinal column
ord_pip=make_pipeline(impute_na_ord,ord_encoder,ord_scaler)
ord_pip


Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='NA', strategy='constant')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['NA', 'Small', 'Medium',
                                             'High']])),
                ('standardscaler', StandardScaler())])

In [32]:
#Create a tuple for each transformer
ord_tuple=('ordinal',ord_pip,ord_cols)
ord_tuple

('ordinal',
 Pipeline(steps=[('simpleimputer',
                  SimpleImputer(fill_value='NA', strategy='constant')),
                 ('ordinalencoder',
                  OrdinalEncoder(categories=[['NA', 'Small', 'Medium',
                                              'High']])),
                 ('standardscaler', StandardScaler())]),
 ['Outlet_Size'])

###Create preprocessing object for nominal columns

In [33]:
#define nominal columns
ohe_cols=X_train.select_dtypes('object').drop(columns=ord_cols).columns
ohe_cols

Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [34]:
#Create OneHotEncoder
ohe_encoder=OneHotEncoder(sparse_output=False,handle_unknown='ignore')

In [35]:
#Create a tuple for each transformer
ohe_tuple=('categories',ohe_encoder,ohe_cols)
ohe_tuple

('categories',
 OneHotEncoder(handle_unknown='ignore', sparse_output=False),
 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
        'Outlet_Location_Type', 'Outlet_Type'],
       dtype='object'))

## Create column transformer

In [36]:
#Creat column transformer with tuples
preprocessor=ColumnTransformer([num_tuple,ord_tuple,ohe_tuple],verbose_feature_names_out=False)
preprocessor

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='NA',
                                                                strategy='constant')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['NA',
                                                                              'Small',
                                                                              'Medium',
                                                                              'High']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Outlet_Size']),
                                ('categories',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object'))],
                  verbose_feature_names_out=False)

In [37]:
type(preprocessor)

sklearn.compose._column_transformer.ColumnTransformer

# Modeling

## Build a linear regression model to predict sales

In [38]:
#Import packages
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [42]:
#Model Evaluate function

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = mean_squared_error(y_true, y_pred, squared=False)
  r_squared = r2_score(y_true, y_pred)
  if verbose == True:

    header = "-"*60
    print(header, f"Regression Metrics: {label}", header, sep='\n')
    print(f"- MAE = {mae:,.3f}")
    print(f"- MSE = {mse:,.3f}")
    print(f"- RMSE = {rmse:,.3f}")
    print(f"- R^2 = {r_squared:,.3f}")
  if output_dict == True:
      metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
      return metrics

def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False):

  y_train_pred = reg.predict(X_train)

  results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label='Training Data')
  print()
  y_test_pred = reg.predict(X_test)
  results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label='Test Data' )

  if output_frame:
    results_df = pd.DataFrame([results_train,results_test])
    results_df = results_df.set_index('Label')
    results_df.index.name=None
    return results_df.round(3)


In [43]:
#Build a linear regression model
lin_reg=LinearRegression()
linreg_pip=make_pipeline(preprocessor,lin_reg)

In [44]:
#Fit model onto train data
linreg_pip.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                                 ('ordinal',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=...
                                                                   OrdinalEncoder(categories=[['NA',
                                                                                               'Small',
                                                                                               'Medium',
                                                                                               'High']])),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Outlet_Size']),
                                                 ('categories',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('linearregression', LinearRegression())])

###Custom evaluation function to get the metrics for the model

In [45]:
#Evaluate the model
evaluate_regression(linreg_pip,X_train,y_train,X_test,y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 847.120
- MSE = 1,297,558.548
- RMSE = 1,139.104
- R^2 = 0.562

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 804.048
- MSE = 1,194,230.406
- RMSE = 1,092.808
- R^2 = 0.567


### Compare the training vs. testing data R-squared values

The linear regression model preformed better on test data(R2=0.567) than train data(R2=0.562). R2 scores for the test data and train data is very close and can only explain about 56% of the variation in target 'price'. Therefore the model is a little underfit.

##Build a Random Forest model to predict sales

In [46]:
#Build a Random Forest model
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(random_state=42)
rf_pip=make_pipeline(preprocessor,rf)

In [47]:
#Fit on train data
rf_pip.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                                 ('ordinal',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=...
                                                                                               'High']])),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Outlet_Size']),
                                                 ('categories',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object'))],
                                   verbose_feature_names_out=False)),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

###Custom evaluation function to get the metrics for the model

In [48]:
evaluate_regression(rf_pip,X_train,y_train,X_test,y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 296.120
- MSE = 182,642.045
- RMSE = 427.366
- R^2 = 0.938

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 767.081
- MSE = 1,216,212.220
- RMSE = 1,102.820
- R^2 = 0.559


###Compare the training vs. test R-squared values

The default Random Forest model perform very well on the train data (R2=0.938) but poorly on test data(R2=0.559). The model is overfit and need tuning.

###Compare this model's performance to the linear regression model

By comparing the test data R^2 scores, linear regression model (R^2=0.567) has higer R^2 than random forest model (R^2=0.559)

##Use GridSearchCV to tune the Random Forest model

In [49]:
# Parameters for tuning
rf_pip.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('numeric',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
          'Outlet_Establishment_Year'],
         dtype='object')),
                                   ('ordinal',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(fill_value='NA',
                                                                   strategy='constant')),
                                                    ('ordinalencoder',
                                                     OrdinalEncoder(categories=[['

In [50]:
#Define param grid try out differnt options
param={'randomforestregressor__max_depth':[1,5,15,25],
       'randomforestregressor__n_estimators':[50,100,200],
       'randomforestregressor__min_samples_leaf':[1,2,3,4],
       'randomforestregressor__oob_score': [False,True]}



In [51]:
#Instantiate the gridsearch
grid_search=GridSearchCV(rf_pip,param,n_jobs=-1,cv=3,verbose=1)

In [52]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')),
                                                                        ('ordinal',
                                                                         Pipeline(steps=[('simpleimpute...
      dtype='object'))],
                                                          verbose_feature_names_out=False)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_depth': [1, 5, 15, 25],
                         'randomforestregressor__min_samples_leaf': [1, 2, 3,
                                                                     4],
                         'randomforestregressor__n_estimators': [50, 100, 200],
                         'randomforestregressor__oob_score': [False, True]},
             verbose=1)

In [53]:
#Check for best parameters
grid_search.best_params_

{'randomforestregressor__max_depth': 5,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__n_estimators': 100,
 'randomforestregressor__oob_score': False}

###Fit and evaluate a final best model on the entire training set


In [54]:
best_model=grid_search.best_estimator_

In [55]:
evaluate_regression(best_model,X_train,y_train,X_test,y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 755.408
- MSE = 1,152,598.974
- RMSE = 1,073.592
- R^2 = 0.611

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 728.426
- MSE = 1,096,412.715
- RMSE = 1,047.097
- R^2 = 0.603


###Compare your tuned model to your default Random Forest

After the tuning of the model, the new tuned model performs better with test data. We see an increase of R2 from 0.559 to 0.603

## Evaluation

Overall I would recommend using the tuned random forest model for prediction because the model has a higher R2 value on test data(R^2=0.603 rf > R^2=0.559 lr) and the model is less underfit than the default random forest model and less overfit than the linear regression model.

The tuned random forest model on test data has a R-squared value of 0.603 indicating that about 60.3% of the variance in 'Outlet Sales' can be explained by the features we're using in our model. The model does a reasonably good job at predicting outlet sales based on the features we provided. However, there are still other factors not included in our model that affect the target. So, while it's decent, there's room for improvement.

The tuned random forest model on test data has a RMSE of 1,047.097 indicating on average our prediction is off by about $1,047.

RMSE tends to penalize larger errors more. Test data has a RMSE of 1,047.097 that is higher than the MAE of 728.426 suggests that there might be some predictions that are quite far off. RMSE delivers more insight regarding about the large error predictions.

The R2 scores for both test and train data of tuned random forest model is very close and model can explain around 60% of the varibility in Outlet sales with features provided. This indicates that the model has decent performance with little underfitting for improvment.